In [2]:
## Naive validation

In [1]:
from pydantic import BaseModel, ConfigDict


class User(BaseModel):
    id: int
    name: str = 'Jane Doe'

    model_config = ConfigDict(str_max_length=10)

In [2]:
user = User(id=123)

In [5]:
assert user.name == 'Jane Doe'  
assert user.id == 123  
assert isinstance(user.id, int)

In [3]:
from pydantic import ValidationError

try:
    user2 = User(id=123, name='Pizzaman1996')
    assert False
except ValidationError as e:
    print(e)
    

1 validation error for User
name
  String should have at most 10 characters [type=string_too_long, input_value='Pizzaman1996', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_too_long


In [4]:
from pydantic import BaseModel


class Model(BaseModel):
    a: int
    b: float
    c: str # it is populated after casting, may cause information loss


print(Model(a=3.000, b='2.72', c=b'binary data').model_dump())

{'a': 3, 'b': 2.72, 'c': 'binary data'}


In [5]:
from pydantic import BaseModel


class Model(BaseModel):
    a: int
    b: float
    c: bytes  # more explicit
    items: list[int] # just another casting here, may leed to poor validation performance


print(Model(a=3.000, b='2.72', c=b'binary data', items=(1,2,3)).model_dump())

{'a': 3, 'b': 2.72, 'c': b'binary data', 'items': [1, 2, 3]}


## Use of extras

In [6]:
from pydantic import BaseModel


class Model1(BaseModel):
    x: int

class Model2(BaseModel):
    x: int

    model_config = ConfigDict(extra='allow') # 'ignore' 'forbid' 'allow'

m1 = Model1(x=1, y='a')
assert m1.model_dump() == {'x': 1} # extra argument is swallowed
m2 = Model2(x=1, y='a')
assert m2.model_dump() == {'x': 1, 'y': 'a'} # extra argument is not swallowed here

## Nested models

In [7]:
from pydantic import BaseModel


class Foo(BaseModel):
    count: int
    size: float | None = None


class Bar(BaseModel):
    apple: str = 'x'
    banana: str = 'y'


class Spam(BaseModel):
    foo: Foo
    bars: list[Bar]


m = Spam(foo={'count': 4}, bars=[{'apple': 'x1'}, {'apple': 'x2'}])
print(m)
"""
foo=Foo(count=4, size=None) bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')]
"""
print(m.model_dump())
"""
{
    'foo': {'count': 4, 'size': None},
    'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}],
}
"""

foo=Foo(count=4, size=None) bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')]
{'foo': {'count': 4, 'size': None}, 'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}]}


"\n{\n    'foo': {'count': 4, 'size': None},\n    'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}],\n}\n"

## SQL Alchemy interaction

In [11]:
from typing import Annotated

from sqlalchemy import ARRAY, String
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column

from pydantic import BaseModel, ConfigDict, StringConstraints


class Base(DeclarativeBase):
    pass


class CompanyOrm(Base):
    __tablename__ = 'companies'

    id: Mapped[int] = mapped_column(primary_key=True, nullable=False)
    public_key: Mapped[str] = mapped_column(
        String(20), index=True, nullable=False, unique=True
    )
    domains: Mapped[list[str]] = mapped_column(ARRAY(String(255)))


class CompanyModel(BaseModel):
    model_config = ConfigDict(from_attributes=True)

    id: int
    public_key: Annotated[str, StringConstraints(max_length=20)]
    domains: list[Annotated[str, StringConstraints(max_length=255)]]


co_orm = CompanyOrm(
    id=123,
    public_key='foobar',
    domains=['example.com', 'foobar.com'],
)
print(co_orm)
#> <__main__.CompanyOrm object at 0x0123456789ab>
co_model = CompanyModel.model_validate(co_orm)
print(co_model)
#> id=123 public_key='foobar' domains=['example.com', 'foobar.com']

id=123 public_key='foobar' domains=['example.com', 'foobar.com']


In [8]:
from pydantic import BaseModel, ConfigDict


class PetCls:
    def __init__(self, *, name: str, species: str):
        self.name = name
        self.species = species


class PersonCls:
    def __init__(self, *, name: str, age: float = None, pets: list[PetCls]):
        self.name = name
        self.age = age
        self.pets = pets


class Pet(BaseModel):
    model_config = ConfigDict(from_attributes=True)

    name: str
    species: str


class Person(BaseModel):
    model_config = ConfigDict(from_attributes=True)

    name: str
    age: float = None
    pets: list[Pet]


bones = PetCls(name='Bones', species='dog')
orion = PetCls(name='Orion', species='cat')
anna = PersonCls(name='Anna', age=20, pets=[bones, orion])
anna_model = Person.model_validate(anna)
print(anna_model)
# """
# name='Anna' age=20.0 pets=[Pet(name='Bones', species='dog'), Pet(name='Orion', species='cat')]
# """

name='Anna' age=20.0 pets=[Pet(name='Bones', species='dog'), Pet(name='Orion', species='cat')]


In [9]:
from dataclasses import dataclass

@dataclass(frozen=True)
class FooBarModelCls:
    a: str
    b: float

class FooBarModel(BaseModel):
    model_config = ConfigDict(from_attributes=True)

    a: str
    b: float

foobar = FooBarModel.model_validate(FooBarModelCls(a='times', b=2))
print(foobar)

a='times' b=2.0


## Validating data

In [12]:
# model_validate => like silver layer validating parquet, json or csv

from datetime import datetime

from pydantic import BaseModel, ValidationError


class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None = None


m = User.model_validate({'id': 123, 'name': 'James'})
print(m)
#> id=123 name='James' signup_ts=None

try:
    User.model_validate(['not', 'a', 'dict'])
except ValidationError as e:
    print(e)

m = User.model_validate_json('{"id": 123, "name": "James"}')
print(m)
#> id=123 name='James' signup_ts=None

try:
    m = User.model_validate_json('{"id": 123, "name": 123}')
except ValidationError as e:
    print(e)

try:
    m = User.model_validate_json('invalid JSON')
except ValidationError as e:
    print(e)

m = User.model_validate_strings({'id': '123', 'name': 'James'})
print(m)
#> id=123 name='James' signup_ts=None

m = User.model_validate_strings(
    {'id': '123', 'name': 'James', 'signup_ts': '2024-04-01T12:00:00'}
)
print(m)
#> id=123 name='James' signup_ts=datetime.datetime(2024, 4, 1, 12, 0)

try:
    m = User.model_validate_strings(
        {'id': '123', 'name': 'James', 'signup_ts': '2024-04-01'}, strict=True
    )
except ValidationError as e:
    print(e)

id=123 name='James' signup_ts=None
1 validation error for User
  Input should be a valid dictionary or instance of User [type=model_type, input_value=['not', 'a', 'dict'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.12/v/model_type
id=123 name='James' signup_ts=None
1 validation error for User
name
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
1 validation error for User
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='invalid JSON', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invalid
id=123 name='James' signup_ts=None
id=123 name='James' signup_ts=datetime.datetime(2024, 4, 1, 12, 0)
1 validation error for User
signup_ts
  Input should be a valid datetime, invalid datetime separator, expected `T`, `t`, `_` or space [type=datetime_parsing, input_value='

## Model construction is unvalidated

In [13]:
class Sale(BaseModel):
    price: int
    amount: float

some_data = dict(price='Luisa', amount= 'Ciccone')
some_sale = Sale.model_construct(**some_data)
print(some_sale)

price='Luisa' amount='Ciccone'


## Copy with Validation (not mentioned)

In [14]:
from pydantic import BaseModel


class BarModel(BaseModel):
    whatever: int


class FooBarModel(BaseModel):
    banana: float
    foo: str
    bar: BarModel


m = FooBarModel(banana=3.14, foo='hello', bar={'whatever': 123})

print(m.model_copy(update={'banana': 'Zoom'}))
#> banana=0 foo='hello' bar=BarModel(whatever=123)

# normal copy gives the same object reference for bar:
print(id(m.bar) == id(m.model_copy().bar))
#> True
# deep copy gives a new object reference for `bar`:
print(id(m.bar) == id(m.model_copy(deep=True).bar))
#> False

banana='Zoom' foo='hello' bar=BarModel(whatever=123)
True
False


In [ ]:
from pydantic import BaseModel


class BarModel(BaseModel):
    whatever: int


class FooBarModel(BaseModel):
    banana: float
    foo: str
    bar: BarModel

    model_config = ConfigDict(revalidate_instances='always')


m = FooBarModel(banana=3.14, foo='hello', bar={'whatever': 123})

try:
    FooBarModel(**m.model_copy(update={'banana': 'Zoom'}).model_dump())
except ValidationError as e:
    print(e)

1 validation error for FooBarModel
banana
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='Zoom', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/float_parsing


c:\Users\VasilisAnagnostopoul\scoop\apps\python\current\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `float` - serialized value may not be as expected [field_name='banana', input_value='Zoom', input_type=str])
  return self.__pydantic_serializer__.to_python(


## Generic types

In [18]:
from pydantic import BaseModel, ValidationError


class DataModel(BaseModel):
    number: int


class Response[DataT](BaseModel):  
    data: DataT  


print(Response[int](data=1))
#> data=1
print(Response[str](data='value'))
#> data='value'
print(Response[str](data='value').model_dump())
#> {'data': 'value'}

data = DataModel(number=1)
print(Response[DataModel](data=data).model_dump())
#> {'data': {'number': 1}}
try:
    Response[int](data='value')
except ValidationError as e:
    print(e)
    """
    1 validation error for Response[int]
    data
      Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='value', input_type=str]
    """

data=1
data='value'
{'data': 'value'}
{'data': {'number': 1}}
1 validation error for Response[int]
data
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='value', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


## Named-tuple styled dynamic model creation

In [17]:
from pydantic import BaseModel, create_model

DynamicFoobarModel = create_model('DynamicFoobarModel', foo=str, bar=(int, 123))

# Equivalent to:


class StaticFoobarModel(BaseModel):
    foo: str
    bar: int = 123

try:
    DynamicFoobarModel(foo='value', bar='key')
except ValidationError as e:
    print(e)    

1 validation error for DynamicFoobarModel
bar
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='key', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


In [18]:
# But this works !!!!!
DynamicFoobarModel(foo='deathstar', bar='666')

DynamicFoobarModel(foo='deathstar', bar=666)

In [21]:
# This works like R

from pydantic import BaseModel, create_model


class FooModel(BaseModel):
    foo: str
    bar: int = 123


BarModel = create_model(
    'BarModel',
    apple=(str, 'russet'),
    banana=(str, 'yellow'),
    __base__=FooModel,
)
print(BarModel)
#> <class '__main__.BarModel'>
print(BarModel.model_fields.keys())
#> dict_keys(['foo', 'bar', 'apple', 'banana'])

<class '__main__.BarModel'>
dict_keys(['foo', 'bar', 'apple', 'banana'])


In [22]:
from pydantic import ValidationError, create_model, field_validator

############# Very old school style, but works!

def alphanum(cls, v):
    assert v.isalnum(), 'must be alphanumeric'
    return v


validators = {
    'username_validator': field_validator('username')(alphanum)  
}

UserModel = create_model(
    'UserModel', username=(str, ...), __validators__=validators
)

user = UserModel(username='scolvin')
print(user)
#> username='scolvin'

try:
    UserModel(username='scolvi%n')
except ValidationError as e:
    print(e)
    """
    1 validation error for UserModel
    username
      Assertion failed, must be alphanumeric [type=assertion_error, input_value='scolvi%n', input_type=str]
    """

username='scolvin'
1 validation error for UserModel
username
  Assertion failed, must be alphanumeric [type=assertion_error, input_value='scolvi%n', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/assertion_error


## Convenience, the root model, a wrapper type

In [23]:
from pydantic import RootModel

Pets = RootModel[list[str]]
PetsByName = RootModel[dict[str, str]]


print(Pets(['dog', 'cat']))
#> root=['dog', 'cat']
print(Pets(['dog', 'cat']).model_dump_json())
#> ["dog","cat"]
print(Pets.model_validate(['dog', 'cat']))
#> root=['dog', 'cat']
print(Pets.model_json_schema())
"""
{'items': {'type': 'string'}, 'title': 'RootModel[list[str]]', 'type': 'array'}
"""

print(PetsByName({'Otis': 'dog', 'Milo': 'cat'}))
#> root={'Otis': 'dog', 'Milo': 'cat'}
print(PetsByName({'Otis': 'dog', 'Milo': 'cat'}).model_dump_json())
#> {"Otis":"dog","Milo":"cat"}
print(PetsByName.model_validate({'Otis': 'dog', 'Milo': 'cat'}))
#> root={'Otis': 'dog', 'Milo': 'cat'}
PetsByName.model_validate({'Otis': 'dog', 'Milo': 'cat'}).root

root=['dog', 'cat']
["dog","cat"]
root=['dog', 'cat']
{'items': {'type': 'string'}, 'title': 'RootModel[list[str]]', 'type': 'array'}
root={'Otis': 'dog', 'Milo': 'cat'}
{"Otis":"dog","Milo":"cat"}
root={'Otis': 'dog', 'Milo': 'cat'}


{'Otis': 'dog', 'Milo': 'cat'}

In [24]:
from pydantic import RootModel


class Pets(RootModel):
    root: list[str]

    def __iter__(self):
        return iter(self.root)

    def __getitem__(self, item):
        return self.root[item]


pets = Pets.model_validate(['dog', 'cat'])
print(pets[0])
#> dog
print([pet for pet in pets])
#> ['dog', 'cat']

dog
['dog', 'cat']


## Immutability (faux)

In [19]:
from pydantic import BaseModel, ConfigDict, ValidationError


class FooBarModel(BaseModel):
    model_config = ConfigDict(frozen=True)

    a: str
    b: dict


foobar = FooBarModel(a='hello', b={'apple': 'pear'})

try:
    foobar.a = 'different'
except ValidationError as e:
    print(e)
    """
    1 validation error for FooBarModel
    a
      Instance is frozen [type=frozen_instance, input_value='different', input_type=str]
    """

print(foobar.a)
#> hello
print(foobar.b)
#> {'apple': 'pear'}
foobar.b['apple'] = 'grape'
print(foobar.b)
#> {'apple': 'grape'}

1 validation error for FooBarModel
a
  Instance is frozen [type=frozen_instance, input_value='different', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/frozen_instance
hello
{'apple': 'pear'}
{'apple': 'grape'}


In [20]:
from dataclasses import dataclass

@dataclass(frozen=True)
class FooBarModel2:
    a: str
    b: dict


foobar2 = FooBarModel2(a='hello', b={'apple': 'pear'})
print(foobar2)

foobar2.b['john']='wick'
print(foobar2)

FooBarModel2(a='hello', b={'apple': 'pear'})
FooBarModel2(a='hello', b={'apple': 'pear', 'john': 'wick'})


## Field order

In [27]:
from pydantic import BaseModel, ValidationError


class Model(BaseModel):
    a: int
    b: int = 2
    c: int = 1
    d: int = 0
    e: float


print(Model.model_fields.keys())
#> dict_keys(['a', 'b', 'c', 'd', 'e'])
m = Model(e=2, a=1)
print(m.model_dump())
#> {'a': 1, 'b': 2, 'c': 1, 'd': 0, 'e': 2.0}
try:
    Model(a='x', b='x', c='x', d='x', e='x')
except ValidationError as err:
    error_locations = [e['loc'] for e in err.errors()]

print(error_locations)
#> [('a',), ('b',), ('c',), ('d',), ('e',)]

dict_keys(['a', 'b', 'c', 'd', 'e'])
{'a': 1, 'b': 2, 'c': 1, 'd': 0, 'e': 2.0}
[('a',), ('b',), ('c',), ('d',), ('e',)]


## Ignore class variables (and private ones, starting with underscore)

In [21]:
## class variables are properly ignored

from typing import ClassVar

from pydantic import BaseModel


class Model(BaseModel):
    x: ClassVar[int] = 1

    y: int = 2


m = Model()
print(m)
#> y=2
print(Model.x)
#> 1

y=2
1


## Signatures

In [29]:
import inspect

from pydantic import BaseModel, Field


class FooModel(BaseModel):
    id: int
    name: str = None
    description: str = 'Foo'
    apple: int = Field(alias='pear')


print(inspect.signature(FooModel))
#> (*, id: int, name: str = None, description: str = 'Foo', pear: int) -> None

(*, id: int, name: str = None, description: str = 'Foo', pear: int) -> None


In [30]:
import inspect

from pydantic import BaseModel


class MyModel(BaseModel):
    id: int
    info: str = 'Foo'

    def __init__(self, id: int = 1, *, bar: str, **data) -> None:
        """My custom init!"""
        super().__init__(id=id, bar=bar, **data)


print(inspect.signature(MyModel))
#> (id: int = 1, *, bar: str, info: str = 'Foo') -> None

(id: int = 1, *, bar: str, info: str = 'Foo') -> None


## Attribute copies

In [31]:
from pydantic import BaseModel


class C1:
    arr = []

    def __init__(self, in_arr):
        self.arr = in_arr


class C2(BaseModel):
    arr: list[int]


arr_orig = [1, 9, 10, 3]


c1 = C1(arr_orig)
c2 = C2(arr=arr_orig)
print(f'{id(c1.arr) == id(c2.arr)=}')
#> id(c1.arr) == id(c2.arr)=False

id(c1.arr) == id(c2.arr)=False
